In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
#from pymantic import sparql

In [2]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [22]:
# Load a datafile
# filepath = "/media/sf_VBox_Shared/CaseLaw/metadata/turtle/meta_1000.ttl"
# sparql.method = 'POST'
# sparql.setQuery('LOAD <file://{}>'.format(filepath))
# res = sparql.query()

In [3]:
#Make a query
queryString = "select (COUNT(*) as ?count) {?s ?p ?o}"
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
ret = sparql.query()
ret.convert()

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '115359'}}]}}

In [4]:
queryString = "prefix dcterm: <http://purl.org/dc/terms/> select (COUNT(*) as ?count) {?s dcterm:type <http://psi.rechtspraak.nl/uitspraak>}"
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
ret = sparql.query()
ret.convert()

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '16389'}}]}}

In [5]:
query_allnodes1 = """prefix dcterm: <http://purl.org/dc/terms/>
select ?id ?title ?creator ?date ?subject ?abstract 
where {
  ?id dcterm:references ?o1.
  optional { ?id dcterm:creator ?creator}.
   optional { ?id dcterm:abstract ?abstract}.
   optional { ?id dcterm:subject ?subject}.
   optional { ?id dcterm:date ?date}.
   optional { ?id dcterm:title ?title}
  
}
"""
sparql.setQuery(query_allnodes1)
sparql.setReturnFormat(JSON)
ret = sparql.query()
allnodes1 = ret.convert()

In [6]:
query_allnodes2 = """prefix dcterm: <http://purl.org/dc/terms/>
select ?id ?title ?creator ?date ?subject ?abstract 
where {
  ?o dcterm:references ?id.
 optional { ?id dcterm:creator ?creator}.
   optional { ?id dcterm:abstract ?abstract}.
   optional { ?id dcterm:subject ?subject}.
   optional { ?id dcterm:date ?date}.
   optional { ?id dcterm:title ?title}
}
"""
sparql.setQuery(query_allnodes2)
sparql.setReturnFormat(JSON)
ret = sparql.query()
allnodes2 = ret.convert()

In [7]:
query_allreferences = """prefix dcterm: <http://purl.org/dc/terms/>
select ?from ?to 
where {
  ?from dcterm:references ?to.
}
"""
sparql.setQuery(query_allreferences)
sparql.setReturnFormat(JSON)
ret = sparql.query()
allreferences = ret.convert()

In [8]:
references = [{'source': d['from']['value'], 'target': d['to']['value']} for d in allreferences['results']['bindings']]

In [9]:
nodes = []
unique_ids = []
for d in allnodes1['results']['bindings'] + allnodes2['results']['bindings']:
    if d['id']['value'] not in unique_ids:
        dout = {}
        for var in allnodes1['head']['vars']:
            dout[var] = d.get(var,{'value':''})['value']
        nodes.append(dout)
        unique_ids.append(d['id']['value'] )

In [10]:
ids_ref = set([d['source'] for d in references] + [d['target'] for d in references])
ids_nodes = set([d['id'] for d in nodes])

In [45]:
print(len(ids_ref), len(ids_nodes), len(nodes))

156 156 156


In [46]:
import json
with open('/media/sf_VBox_Shared/CaseLaw/graphs/wgaans.json', 'w') as outfile:
    json.dump({'nodes': nodes, 'links': references}, fp=outfile)

In [13]:
import json
#Set id for sigma
for r in references:
    r['id'] = r['source'] + '_' + r['target']

with open('/media/sf_VBox_Shared/CaseLaw/graphs/wgaans_sigma.json', 'w') as outfile:
    json.dump({'nodes': nodes, 'edges': references}, fp=outfile)